<i>STATUS: Draft<i>

In [3]:
import numpy as np
import sympy as sp
from IPython.display import HTML, IFrame
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
from IPython.display import Image
import sys
import HTM_Code as hc
import pandas as pd
import calendar

In the last notebook, we built our first encoder. Admittedly, it was very simple. It just allows us to create integers and provide them to, and provide back indice location. It provided a little extra functionality so it could track previous, and comparison. And we established some rules that we would like to see in place. 

We also discussed what similiarity metrics in such an environment might look like, based on the rules. Its important to emphasie that we have really conflated the idea of semantic meaning, with distance, which is problematic, and we will need to take a deeper look at what it means to really encode something semantically. 

Finally, the Encoder Class that was introduced in the last notebook is also included in the HTM_Code.py file for us to use


<div style="background:#99ddff; color:black; padding: 10px">
<b>Add to these notes:</b>

I want to keep these notes in HTML so I don't have to host on a server, but a great exceriise is to use the ipython widgets to experience this like it happens in the video. 
</div>

In [4]:
IFrame("https://www.youtube.com/embed/PTYlge2K1G8", width=600, height=300)

This episode is about how we might build Date and Time Encoder, but the way I like to think of this is . But the idea behind this is really that join multiple encoders togehter. For example, I might be interested in an encoder that takes 365 values, one for each day of the year, or just 2 values (one for whether it is weekend or not the weekend), 4 values (telling me which of the seasons of the year it is), or a minute encoder (telling which minute of the day). 

Under the hood, these are each no different from the encoders that we were already looking at. Each value that would stored is simply and SDR with active bits and an SDR size. 

I could join each of these encoders together (by simply concatentating the arrays), and I would in principle have an encoder that tracks the meaning of theach these, and I could feed into it a date and time, and see thier differen they are. 

To explore this further, let's build another class: a multi enncoder

To explore this, let's use some data. There is some really interesting data that will turn up in episode's 7 and 8 in the context of the Spatial Pooler, that has some interesting info, but for now, let's use Baltimore Crime Data. This data has nice coverage across a number of data points, descriptive names, some categorical variables, the footprint isn't too big but it gives us a nice sample of 96k records

Information available <a href="https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about">https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about</a>


In [5]:
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96976 entries, 0 to 96975
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowID            96976 non-null  int64  
 1   CrimeDateTime    96976 non-null  object 
 2   CrimeTime        96969 non-null  object 
 3   CrimeCode        96976 non-null  object 
 4   Location         96399 non-null  object 
 5   Description      96976 non-null  object 
 6   Inside_Outside   94136 non-null  object 
 7   Weapon           19832 non-null  object 
 8   Post             96797 non-null  object 
 9   District         96797 non-null  object 
 10  Neighborhood     96790 non-null  object 
 11  Latitude         96921 non-null  float64
 12  Longitude        96921 non-null  float64
 13  GeoLocation      96976 non-null  object 
 14  Premise          94007 non-null  object 
 15  VRIName          11491 non-null  object 
 16  Total_Incidents  96976 non-null  int64  
 17  HashedRecord

The first thing I want to do is fix some formatting and some derived data, so we can see day, season which will show us more about our encoder, and how we can use it.

In [6]:
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.CrimeDateTime = df.CrimeDateTime.str.slice(0, -8)
df.CrimeDateTime= pd.to_datetime(df.CrimeDateTime)
df['weekdayCodeWhenEventReported'] = [d.weekday() for d in df.CrimeDateTime]
df['monthCodeWhenEventReported'] = df['CrimeDateTime'].dt.month
df['seasonCodeWhenEventReported'] = (df['CrimeDateTime'].dt.month - 1) % 4
df['isWeekend'] = np.where(df.weekdayCodeWhenEventReported > 4, True, False)
df = df.drop('VRIName', axis=1)
df = df.drop('HashedRecord', axis=1)
df = df.drop('ObjectId', axis=1)

In [7]:
df.sample(n = 5).transpose()

,70831,23944,19406,76330,28235
RowID,254401,207013,202723,259451,211400
CrimeDateTime,2015-07-24 00:00:00,2016-07-20 00:00:00,2016-08-21 00:00:00,2015-06-20 00:00:00,2016-06-18 00:00:00
CrimeTime,9:45:00,21:00:00,13:10:00,0:20:00,21:00:00
CrimeCode,4E,5E,6C,3AJF,6D
Location,4700 FREDERICK AVE,2800 GREENMOUNT AVE,400 E NORTH AVE,400 FONT HILL AVE,6200 BROWN AVE
Description,COMMON ASSAULT,BURGLARY,LARCENY,ROBBERY - CARJACKING,LARCENY FROM AUTO
Inside_Outside,I,I,I,O,O
Weapon,NaN,NaN,NaN,FIREARM,NaN
Post,822,513,311,832,234
District,SOUTHWEST,NORTHERN,EASTERN,SOUTHWEST,SOUTHEAST


There is also not really a sense of a Scalar Number in this data, and it would nice to see how that plays out in an encoder which I want to use, so for our purposes I also am going to create complete fictional variable which I will call 'investigativeResourcesRequired', and I will make that correlated to description

In [8]:
descriptionValues = pd.DataFrame(pd.value_counts(df.Description)).index.values
startingVal = 20
intervalVal = 5
sdForDistribution = 4

frameList = []

for i in range(len(descriptionValues)):
    startingVal = startingVal + intervalVal
    numberOfValuesToCreate = len(df[df.Description == descriptionValues[i]])
    randomDistributedValues = np.random.normal(startingVal, sdForDistribution, numberOfValuesToCreate)
    newFrame = df[df.Description == descriptionValues[i]]
    newFrame = newFrame.reset_index()
    newFrame.drop('index', axis=1)
    newFrame['investigativeResourcesRequired'] = randomDistributedValues
    frameList.append(newFrame)

df = pd.concat(frameList, ignore_index=True)
df = df.drop('index', axis=1)

Let's have a look at a sample of our final dataset

In [136]:
df.sample(n=5).transpose()

,35820,53086,63586,42307,42610
RowID,276535,198977,273972,222138,224496
CrimeDateTime,2015-02-02 00:00:00,2016-09-16 00:00:00,2015-03-02 00:00:00,2016-04-01 00:00:00,2016-03-12 00:00:00
CrimeTime,21:00:00,16:00:00,23:30:00,13:40:00,2:17:00
CrimeCode,5A,6D,6D,4E,4E
Location,2400 E MONUMENT ST,2800 DILLON ST,500 N COLLINGTON AVE,700 E 23RD ST,2500 W FAIRMOUNT AVE
Description,BURGLARY,LARCENY FROM AUTO,LARCENY FROM AUTO,COMMON ASSAULT,COMMON ASSAULT
Inside_Outside,I,O,NaN,I,NaN
Weapon,NaN,NaN,NaN,NaN,NaN
Post,333,214,321,312,835
District,EASTERN,SOUTHEAST,EASTERN,EASTERN,SOUTHWEST


What we are left with is a bunch of differnt types of records: some numerical records on a range, as well as some categorical records. There is some interesting data here. I will leave it to you to explore, but the point is it raises some interesting challenges to what we mean by semantic similarity, and how we would put this in some encoder that captures the information as a whole 

Let's create 4 encoders from this data: 
- a timeOfDay encoder 
- a dayOfWeekEncoder 
- an isWeekend encoder
- a season encoder and an 

We will set a global size for each 1, and 8 values

In [239]:
class Encoder:
    def __init__(self, bit_space_size = None,
                number_of_bits_used_to_encode_value = None,
                min_val = None,
                max_val = None,
                resolution = None,
                is_randomly_distributed = None,
                clip_values_outside_range = None,
                is_periodic = None):

        self.bit_space_size = bit_space_size
        self.number_of_bits_used_to_encode_value = number_of_bits_used_to_encode_value
        self.clip_values_outside_range = clip_values_outside_range
        self.is_periodic = is_periodic
        self.is_randomly_distributed = is_randomly_distributed

        self.resolution = resolution
        self.uniqueness = 1
        self.permittedSimiliarityBetweenEncodings = .90 
        self.similiarityIsPercentage = True
        self.min_value_to_encode = min_val
        self.max_value_to_encode = max_val
        self.max_bit_space_value = bit_space_size
        self.min_bit_space_value = 0
        self.buckets = []
        self.encoded_values_bit_locations = []
        self.offset_for_array_indice = 1
        self.encodedValues = {}
        
        self.bucket_capacity = self.computeBucketCapacity()
        
        if self.is_randomly_distributed:
            self.initial_encoding = np.array(hc.create_randomised_sdr(self.bit_space_size, self.number_of_bits_used_to_encode_value))

            self.encoded_values_and_bit_locations = {str(self.min_value_to_encode):self.initial_encoding}
            self.buckets.append(self.min_value_to_encode)
            self.encoded_values_bit_locations.append(np.array(self.initial_encoding))
        
    def get_summary(self):
        print("----------------- SUMMARY -------------------------")
        print("|L3| Bit Space Size: ", self.bit_space_size)
        print("|L4| Number of bits to be used when encoding each value:", self.number_of_bits_used_to_encode_value)
        print("|L5| Range of values that can be encoded: From ", self.min_value_to_encode, ' to ', self.max_value_to_encode)
        print("|L6| Number of buckets available in bit space:", float(self.bucket_capacity))
        print("|L1| Encode periodically: ", self.is_periodic)
        print("|L1| Values are encoded as are randomly distributed arrays: ", self.is_randomly_distributed)
        print("|L1| Resolution: ", self.resolution)
        print("|L1| Similiarity between encodings: ", self.uniqueness)
        print("|L2| Values outside range will to be clipped: ",self.clip_values_outside_range)
        print("|L7| Encoded values bit locations:\n ", self.encoded_values_bit_locations)
        print("|L8| Buckets", self.buckets)
        print("----------------------------------------------------")

        
    def computeBucketCapacity(self):
        """The purpose of this function is compute bucket capacity. Bucket capacity"""
        if self.is_randomly_distributed:
            return(np.floor(sp.binomial(self.bit_space_size, self.number_of_bits_used_to_encode_value) / self.resolution))
        else:
            return(np.floor((self.bit_space_size - self.number_of_bits_used_to_encode_value + 1)/ self.resolution))

    def makeRandomMove(self):
        
        newBitLocationsMayHaveClashWithExistingBitLocations = True
        
        while(newBitLocationsMayHaveClashWithExistingBitLocations):
            random_bit_index_to_move = np.random.randint(0, self.number_of_bits_used_to_encode_value, 1)[0]
            random_direction_to_move = np.random.randint(0, 2, 1)

            next_sdr = self.encoded_values_bit_locations[-1].copy()
            value = next_sdr[random_bit_index_to_move]

            if random_direction_to_move == 1:
                value = next_sdr[random_bit_index_to_move] + 1
            else: 
                value = next_sdr[random_bit_index_to_move] - 1

            if value > self.max_bit_space_value:
                if self.is_periodic:
                    value = self.min_bit_space_value
                else:
                    value = value - 2
            elif value < self.min_bit_space_value:
                if self.is_periodic:
                    value = self.max_bit_space_value
                else:
                    value = value + 2

            next_sdr[random_bit_index_to_move] = value
            newBitLocationsMayHaveClashWithExistingBitLocations = np.any(np.all(next_sdr == self.encoded_values_bit_locations, axis=1))
            
        return(next_sdr)
    
    def createBucketsForRandomlyEncodedValues(self, iterations_needed):
        for i in range(0, iterations_needed):
            next_sdr = self.makeRandomMove()
            self.encoded_values_bit_locations.append(next_sdr.copy())
            self.buckets.append(np.array(self.buckets[-1] + 1))
            self.encoded_values_and_bit_locations[str(self.buckets[-1])] = next_sdr.copy()

    
    def checkWhichBucket(self, valueChoice):
        bucket = np.floor(valueChoice / self.resolution)
        return(int(bucket))
        
    def updateEncodedValuesDictionary(self):
        for i in range(len(self.buckets)):

            self.encodedValues[str(self.buckets[i])] = {"bitLocations": self.encoded_values_bit_locations[i],
                                                   "encodedValues": np.arange(start=self.buckets[i], stop=self.buckets[i] + self.resolution, step=1)}
                    
        print(self.encodedValues)
    
    def encode_value_in_bit_space(self, value_choice):

        bucket = self.checkWhichBucket(value_choice)
        print("Value", value_choice, "will be encoded into bucket ", bucket)
        value_choice = bucket
        
        print("\nEncoding the value ->", value_choice)
        
        if (value_choice < self.min_value_to_encode or value_choice > self.max_value_to_encode) and not self.is_periodic:
            if self.clip_values_outside_range:
                valuePriorToClipping = value_choice
                if value_choice < self.min_value_to_encode:
                    value_choice = self.min_value_to_encode
                else:
                    value_choice = self.max_value_to_encode
                print("The value of: ", valuePriorToClipping, "has been clipped to ->", value_choice)
            else:
                print("Not a valid choice (clipValuesOutsideRange has been set to False)")
                return


        if self.is_randomly_distributed:
            if (value_choice < self.buckets[-1]):
                print("There is a bucket already created for the value", value_choice, "-> ", 
                      self.encoded_values_and_bit_locations[str(value_choice)])
                return
            
            buckets_needed_to_encode_value = value_choice - self.buckets[-1]
            print("Current number of encoded values: " , len(self.buckets))
  
            print("Value encoded in first bucket: ", self.min_value_to_encode)
            print("Number of additional buckets required to accomodate the value choice of", value_choice, ": ", buckets_needed_to_encode_value)
            self.createBucketsForRandomlyEncodedValues(buckets_needed_to_encode_value)
        
        else:
            window = [value_choice, value_choice + self.number_of_bits_used_to_encode_value]

            if self.is_periodic:

                inputValues = np.arange(0,self.bit_space_size)
                all_values =  []

                for eachValue in range(window[0], window[1]):
                    all_values.append(inputValues[eachValue % self.bit_space_size])
    
                self.encoded_values_bit_locations.append(all_values)
                self.buckets.append(value_choice)

            else:
 
                all_values = np.arange(window[0], window[1])
                self.encoded_values_bit_locations.append(all_values)
                self.buckets.append(value_choice)
   
           

 

In [240]:
bit_space_size_choice = 30
number_of_bits_used_to_encode_value_choice = 6
e1 = Encoder(bit_space_size = bit_space_size_choice,
                number_of_bits_used_to_encode_value = number_of_bits_used_to_encode_value_choice,
                min_val = 0,
                max_val = 27,
             resolution=3,
            is_randomly_distributed = True,
            clip_values_outside_range = True,
            is_periodic = False)
e1.get_summary()
e1.encode_value_in_bit_space(0)
e1.encode_value_in_bit_space(1)
e1.encode_value_in_bit_space(2)
e1.encode_value_in_bit_space(3)
e1.encode_value_in_bit_space(4)
e1.encode_value_in_bit_space(5)
e1.encode_value_in_bit_space(60)

e1.get_summary()
e1.buckets
#e1.encoded_values_bit_locations
e1.updateEncodedValuesDictionary()

----------------- SUMMARY -------------------------
|L3| Bit Space Size:  30
|L4| Number of bits to be used when encoding each value: 6
|L5| Range of values that can be encoded: From  0  to  27
|L6| Number of buckets available in bit space: 197925.0
|L1| Encode periodically:  False
|L1| Values are encoded as are randomly distributed arrays:  True
|L1| Resolution:  3
|L1| Similiarity between encodings:  1
|L2| Values outside range will to be clipped:  True
|L7| Encoded values bit locations:
  [array([ 1,  3, 13, 15, 18, 23])]
|L8| Buckets [0]
----------------------------------------------------
Value 0 will be encoded into bucket  0

Encoding the value -> 0
Current number of encoded values:  1
Value encoded in first bucket:  0
Number of additional buckets required to accomodate the value choice of 0 :  0
Value 1 will be encoded into bucket  0

Encoding the value -> 0
Current number of encoded values:  1
Value encoded in first bucket:  0
Number of additional buckets required to accomodat

In [241]:
e1.encodedValues

{'0': {'bitLocations': array([ 1,  3, 13, 15, 18, 23]),
  'encodedValues': array([0, 1, 2])},
 '1': {'bitLocations': array([ 1,  3, 13, 15, 18, 22]),
  'encodedValues': array([1, 2, 3])},
 '2': {'bitLocations': array([ 2,  3, 13, 15, 18, 22]),
  'encodedValues': array([2, 3, 4])},
 '3': {'bitLocations': array([ 2,  3, 12, 15, 18, 22]),
  'encodedValues': array([3, 4, 5])},
 '4': {'bitLocations': array([ 2,  3, 12, 16, 18, 22]),
  'encodedValues': array([4, 5, 6])},
 '5': {'bitLocations': array([ 2,  3, 12, 17, 18, 22]),
  'encodedValues': array([5, 6, 7])},
 '6': {'bitLocations': array([ 2,  3, 13, 17, 18, 22]),
  'encodedValues': array([6, 7, 8])},
 '7': {'bitLocations': array([ 2,  3, 14, 17, 18, 22]),
  'encodedValues': array([7, 8, 9])},
 '8': {'bitLocations': array([ 2,  3, 14, 17, 19, 22]),
  'encodedValues': array([ 8,  9, 10])},
 '9': {'bitLocations': array([ 2,  4, 14, 17, 19, 22]),
  'encodedValues': array([ 9, 10, 11])},
 '10': {'bitLocations': array([ 2,  4, 13, 17, 19, 22]

In [167]:
timeOfDayEncoder.encode_value_in_bit_space(3)


Encoding the value -> 3
There is a bucket already created for the value 3 ->  [3 4]


Now onto next episode of HTM School

In [6]:
class MultiEncoder:
    def __init__(self):
        self.encoders = []
        self.bit_space_size = None
    def add_encoder(self, encoder):
        self.encoders.append(encoder)
    def join_encoders(self):
        pass

So this is a really powerful idea. We can think about multiple time series unfolding, date time components. Or we could think of symbolic music, track key changes

In [ ]:
def compute_union_and_overlap(SDR1_on_bits, SDR2_on_bits):
    union = list(set(SDR1_on_bits).union(SDR2_on_bits))
    overlap = list(set(SDR1_on_bits).intersection(SDR2_on_bits))
    
    return({"union": union, "overlap": overlap})



This is starting to seem more like we saw in the early notebooks, we can how semantic similairty is affected by noise and subsampling

There is alot more we can do with encoders. Delta encoder, log encoder. A geospatial encoder particularly interesting, enconding values on a sphere, what other geometrical shapes, opens us up to diffent types of geometry adn topology also

Let's look at more encoders: 
https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

IMportant to capture semantics properly

4 principles recall..... eg consider two numbers should be semantically similiar 

Note for encoders, we can encode anything we can put in a relationship of 

we need encoders to have to to incorporate noise and subsamplinig 

Encoding Daa for HTM systems - Purdy


Date Time Encoder

 
